Importando as bibliotecas

In [19]:
import tensorflow as tf
import tensorflow.keras as keras
import sys
import matplotlib.pyplot as plt
import numpy as np

# **Dataset**

In [2]:
sys.path.append('..')

train_dir = '../data/train'
test_dir = '../data/test'

Optamos por setar um tamanho fixo de (224, 244) para as nossas imagens, já que usaremos a arquitetura AlexNet.

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True
)

num_classes = len(train_dataset.class_names)

## **Sobre os dados**

Escolhemos um dataset de classificação multiclasse sobre 10 espécies distintas de macacos:

In [ ]:
print(train_dataset.class_names)

### **Visualizando algumas amostras**

In [5]:
from random import randint
def choose_image(dataset, batch_size):
    idx = randint(0, batch_size-1)
    for images, labels in dataset.take(1):
        sample_image = images[idx].numpy().astype(int)
        sample_label = labels[idx].numpy()
        break
    return sample_image, sample_label

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(10, 6))

for i in range(2):
    for j in range(3):
        img, label = choose_image(train_dataset, 32)
        axs[i][j].imshow(img)
        axs[i][j].set_xlabel(f'species: {label}')

# **Modelo**

Baseado na arquitetura AlexNet

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=(224, 224, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((3, 3), strides=2),
    
    keras.layers.Conv2D(256, (5, 5), padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((3, 3), strides=2),
    
    keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
    keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
    keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D((3, 3), strides=2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation='softmax')
])

Utilizamos o otimizador Adam, a função custo de Sparse Categorical CrossEntropy, e reportamos a métrica da acurácia a cada época.

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# **Treinamento**

In [ ]:
EPOCHS = 200
history = model.fit(train_dataset, validation_data=test_dataset, epochs=EPOCHS)

In [ ]:
model.save('../models/trained_model.h5')

## **Gráfico da função custo**

In [ ]:
plt.title('Curva de aprendizado do modelo')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Treino', 'Validação'])
plt.xlabel('Épocas')
plt.ylabel('Loss')

# **Avaliação do modelo**

In [ ]:
y_true = []
y_pred = []

for images, labels in test_dataset:
    preds = model.predict(images) 
    y_true.extend(labels.numpy()) 
    y_pred.extend(np.argmax(preds, axis=1))

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

## **Acurácia**

In [ ]:
print(f'Acurácia: {accuracy_score(y_pred, y_true):.2f}')

## **Matriz de confusão**

In [ ]:
cm = confusion_matrix(y_pred, y_true, normalize='true')
cm_display = ConfusionMatrixDisplay(cm)

In [ ]:
cm_display.plot()